In [1]:
from __future__ import print_function
from PIL import Image
import os
import os.path
import errno
import numpy as np
import sys
import torch
if sys.version_info[0] == 2:
    import cPickle as pickle
else:
    import pickle

import torch.utils.data as data
from torchvision.datasets.utils import download_url, check_integrity
import torchvision.transforms as tt
from torchvision import datasets, transforms
import torchvision

class CIFAR10(data.Dataset):
    """`CIFAR10 <https://www.cs.toronto.edu/~kriz/cifar.html>`_ Dataset.
    Args:
        root (string): Root directory of dataset where directory
            ``cifar-10-batches-py`` exists.
        train (bool, optional): If True, creates dataset from training set, otherwise
            creates from test set.
        transform (callable, optional): A function/transform that  takes in an PIL image
            and returns a transformed version. E.g, ``transforms.RandomCrop``
        target_transform (callable, optional): A function/transform that takes in the
            target and transforms it.
        download (bool, optional): If true, downloads the dataset from the internet and
            puts it in root directory. If dataset is already downloaded, it is not
            downloaded again.
    """
    base_folder = 'cifar-10-batches-py'
    url = "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"
    filename = "cifar-10-python.tar.gz"
    tgz_md5 = 'c58f30108f718f92721af3b95e74349a'
    train_list = [
        ['data_batch_1', 'c99cafc152244af753f735de768cd75f'],
        ['data_batch_2', 'd4bba439e000b95fd0a9bffe97cbabec'],
        ['data_batch_3', '54ebc095f3ab1f0389bbae665268c751'],
        ['data_batch_4', '634d18415352ddfa80567beed471001a'],
        ['data_batch_5', '482c414d41f54cd18b22e5b47cb7c3cb'],
    ]

    test_list = [
        ['test_batch', '40351d587109b95175f43aff81a1287e'],
    ]

    meta = {
        "filename": "batches.meta",
        "key": "label_names",
        "md5": "5ff9c542aee3614f3951f8cda6e48888",
    }


    def __init__(self, root, train=True,
                 transform=None, target_transform=None,
                 download=False, coarse=False, coarseNumber=None):
        self.root = os.path.expanduser(root)
        self.transform = transform
        self.target_transform = target_transform
        self.train = train  # training set or test set
        self.coarse = coarse

        if download:
            self.download()

        if not self._check_integrity():
            raise RuntimeError('Dataset not found or corrupted.' +
                               ' You can use download=True to download it')

        # now load the picked numpy arrays
        if self.train:
            self.train_data = []
            self.train_labels = []
            self.train_coarse_labels = []
            for fentry in self.train_list:
                f = fentry[0]
                file = os.path.join(self.root, self.base_folder, f)
                fo = open(file, 'rb')
                if sys.version_info[0] == 2:
                    entry = pickle.load(fo)
                else:
                    entry = pickle.load(fo, encoding='latin1')
                self.train_data.append(entry['data'])
                if 'labels' in entry:
                    self.train_labels += entry['labels']
                else:
                    self.train_labels += entry['fine_labels']
                    if self.coarse:
                        self.train_coarse_labels += entry['coarse_labels']
                fo.close()

            new_train_coarse_labels=[]
            new_train_fine_labels=[]
            print("fine train",self.train_labels)
            print(len(self.train_labels))
            print("coarse ",self.train_coarse_labels)
            print(len(self.train_coarse_labels))

            for fine,coarse in zip(self.train_labels, self.train_coarse_labels):
                if coarse==coarseNumber:
                    new_train_coarse_labels.append(coarseNumber)
                    new_train_fine_labels.append(fine)
            print("coarse ",new_train_coarse_labels)
            print("fine ",new_train_fine_labels)
            print(len(new_train_coarse_labels))
            print(len(new_train_fine_labels))
            unique_fine_in_this_coarse = list(set(new_train_fine_labels))
            print("unique fine labels ", str(unique_fine_in_this_coarse)+" in coarse numbah ", coarseNumber)

            self.train_data = np.concatenate(self.train_data)
            self.train_data = self.train_data.reshape((50000, 3, 32, 32))
            self.train_data = self.train_data.transpose((0, 2, 3, 1))  # convert to HWC
        else:
            f = self.test_list[0][0]
            file = os.path.join(self.root, self.base_folder, f)
            fo = open(file, 'rb')
            if sys.version_info[0] == 2:
                entry = pickle.load(fo)
            else:
                entry = pickle.load(fo, encoding='latin1')
            self.test_data = entry['data']

            if 'labels' in entry:
                self.test_labels = entry['labels']
            else:
                self.test_labels = entry['fine_labels']
                if self.coarse:
                    self.test_coarse_labels = entry['coarse_labels']
                new_test_coarse_labels=[]
                new_test_fine_labels=[]
                new_test_data=[]
                print("fine ",entry['fine_labels'])
                print(len(entry['fine_labels']))
                print("coarse ",entry['coarse_labels'])
                print(len(entry['coarse_labels']))
                for data, fine,coarse in zip(self.test_data,self.test_labels, self.test_coarse_labels):
                    if coarse==coarseNumber:
                        new_test_data.append(data)
                        new_test_fine_labels.append(fine)
                        new_test_coarse_labels.append(coarseNumber)
                print("coarse ",new_test_coarse_labels)
                print("fine ",new_test_fine_labels)
                print(len(new_test_coarse_labels))
                print(len(new_test_fine_labels))
                print("============")
                print("data ",new_test_data)
                print(len(new_test_data))
            fo.close()
            self.test_data = self.test_data.reshape((10000, 3, 32, 32))
            self.test_data = self.test_data.transpose((0, 2, 3, 1))  # convert to HWC

    def __getitem__(self, index):
        """
        Args:
            index (int): Index
        Returns:
            tuple: (image, target) where target is index of the target class.
        """
        if self.train:
            img, target = self.train_data[index], self.train_labels[index]
            if self.coarse:
                coarse_target = self.train_coarse_labels[index]
        else:
            img, target = self.test_data[index], self.test_labels[index]
            if self.coarse:
                coarse_target = self.test_coarse_labels[index]

        # doing this so that it is consistent with all other datasets
        # to return a PIL Image
        img = Image.fromarray(img)

        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)
        if not self.coarse:
            return img, target
        else:
            return img, target, coarse_target

    def __len__(self):
        if self.train:
            return len(self.train_data)
        else:
            return len(self.test_data)

    def _check_integrity(self):
        root = self.root
        for fentry in (self.train_list + self.test_list):
            filename, md5 = fentry[0], fentry[1]
            fpath = os.path.join(root, self.base_folder, filename)
            if not check_integrity(fpath, md5):
                return False
        return True

    def download(self):
        import tarfile

        if self._check_integrity():
            print('Files already downloaded and verified')
            return

        root = self.root
        download_url(self.url, root, self.filename, self.tgz_md5)

        # extract file
        cwd = os.getcwd()
        tar = tarfile.open(os.path.join(root, self.filename), "r:gz")
        os.chdir(root)
        tar.extractall()
        tar.close()
        os.chdir(cwd)


class CIFAR100(CIFAR10):
    """`CIFAR100 <https://www.cs.toronto.edu/~kriz/cifar.html>`_ Dataset.
    This is a subclass of the `CIFAR10` Dataset.
    """
    base_folder = 'cifar-100-python'
    url = "https://www.cs.toronto.edu/~kriz/cifar-100-python.tar.gz"
    filename = "cifar-100-python.tar.gz"
    tgz_md5 = 'eb9058c3a382ffc7106e4002c42a8d85'
    train_list = [
        ['train', '16019d7e3df5f24257cddd939b257f8d'],
    ]

    test_list = [
        ['test', 'f0ef6b0ae62326f3e7ffdfab6717acfc'],
    ]


In [2]:
transform_train = tt.Compose([tt.RandomCrop(32, padding=4,padding_mode='reflect'), 
                         tt.RandomHorizontalFlip(), 
                         tt.ToTensor(), 
                         #tt.Normalize(mean,std,inplace=True) 
                         ]
                         )
transform_test = tt.Compose([tt.ToTensor(), 
                             #tt.Normalize(mean,std)
                             ])

In [3]:
batch_size = 400
epochs = 120
max_lr = 0.001
grad_clip = 0.01
weight_decay =0.001
opt_func = torch.optim.Adam

In [4]:
train_data = CIFAR100('./data', train=True,
                 transform=transform_train,
                 download=True, coarse=True, coarseNumber=5)
test_data = CIFAR100('./data', train=False,
                 transform=transform_test,
                 download=True, coarse=True, coarseNumber=5)

train_length = train_data.__len__() # Length training dataset
train_indices = np.arange(train_length)
    
train_loader = torch.utils.data.DataLoader(
                        train_data, 
                        batch_size=batch_size, 
                        num_workers=2,
                        pin_memory=True,
                    )

test_loader = torch.utils.data.DataLoader(
                        test_data, 
                        batch_size=batch_size*2,
                        num_workers=2,
                        pin_memory=True
                    )

Files already downloaded and verified
fine train [19, 29, 0, 11, 1, 86, 90, 28, 23, 31, 39, 96, 82, 17, 71, 39, 8, 97, 80, 71, 74, 59, 70, 87, 59, 84, 64, 52, 42, 64, 8, 17, 47, 65, 21, 22, 81, 11, 24, 84, 78, 45, 49, 97, 56, 76, 11, 90, 89, 78, 73, 14, 87, 9, 71, 6, 47, 20, 98, 47, 36, 55, 72, 43, 51, 35, 83, 33, 27, 53, 92, 50, 15, 89, 36, 18, 89, 46, 33, 42, 39, 64, 75, 38, 23, 42, 66, 77, 49, 18, 46, 15, 35, 69, 95, 83, 75, 99, 73, 93, 55, 39, 4, 97, 61, 93, 51, 69, 56, 84, 59, 64, 94, 4, 11, 33, 68, 38, 20, 33, 34, 32, 46, 53, 88, 67, 70, 64, 53, 64, 8, 96, 87, 30, 20, 30, 66, 19, 76, 87, 52, 62, 35, 63, 40, 4, 99, 63, 74, 53, 26, 95, 48, 27, 33, 29, 39, 79, 32, 46, 64, 28, 85, 32, 82, 78, 39, 54, 28, 66, 65, 72, 21, 64, 62, 72, 0, 44, 7, 12, 19, 11, 31, 61, 79, 45, 81, 79, 98, 43, 46, 67, 80, 68, 74, 48, 81, 94, 86, 69, 39, 73, 2, 46, 49, 63, 43, 14, 49, 68, 65, 41, 37, 45, 36, 21, 77, 37, 39, 8, 9, 62, 86, 39, 19, 54, 39, 28, 11, 89, 90, 90, 79, 66, 81, 21, 79, 40, 29, 22, 13, 2

In [8]:
#Basic EDA with labels for train_data
demo_loader = torch.utils.data.DataLoader(train_data, batch_size=1200)

demo_batch = next(iter(demo_loader))
img, fine_labels, course_labels = demo_batch
fine_labels=fine_labels.numpy()
course_labels=course_labels.numpy()
print(course_labels)
print(fine_labels)

parent_to_child_class={}

for fine, coarse in zip(fine_labels, course_labels):
    if coarse not in parent_to_child_class:
        parent_to_child_class[coarse]=[fine]
    elif fine not in parent_to_child_class[coarse]:
        parent_to_child_class[coarse].append(fine)
        parent_to_child_class[coarse].sort()

print(len(parent_to_child_class.keys()))

count=0
for x in parent_to_child_class.values():
    count+=len(x)
print(count)

print(parent_to_child_class.keys())

[11 15  4 ...  2 14 11]
[19 29  0 ... 54  2 15]
20
100
dict_keys([11, 15, 4, 14, 1, 5, 18, 3, 10, 17, 2, 9, 8, 16, 6, 12, 19, 7, 13, 0])


In [9]:
parent_to_child_class.items()

dict_items([(11, [15, 19, 21, 31, 38]), (15, [27, 29, 44, 78, 93]), (4, [0, 51, 53, 57, 83]), (14, [2, 11, 35, 46, 98]), (1, [1, 32, 67, 73, 91]), (5, [22, 39, 40, 86, 87]), (18, [8, 13, 48, 58, 90]), (3, [9, 10, 16, 28, 61]), (10, [23, 33, 49, 60, 71]), (17, [47, 52, 56, 59, 96]), (2, [54, 62, 70, 82, 92]), (9, [12, 17, 37, 68, 76]), (8, [3, 42, 43, 88, 97]), (16, [36, 50, 65, 74, 80]), (6, [5, 20, 25, 84, 94]), (12, [34, 63, 64, 66, 75]), (19, [41, 69, 81, 85, 89]), (7, [6, 7, 14, 18, 24]), (13, [26, 45, 77, 79, 99]), (0, [4, 30, 55, 72, 95])])

In [10]:
parent_to_child_class[0]

[4, 30, 55, 72, 95]

In [23]:
parent_to_child_class_from_0_to_5={}

for parent, children in parent_to_child_class.items():
    print(parent)
    print(children)
    test_keys = children.copy()
    test_values = [0, 1, 2, 3 ,4]

    # Printing original keys-value lists
    print("Original key list is : " + str(test_keys))
    print("Original value list is : " + str(test_values))

    # using naive method
    # to convert lists to dictionary
    res = {}
    for key in test_keys:
        for value in test_values:
            res[int(key)] = value
            test_values.remove(value)
            break
    parent_to_child_class_from_0_to_5[int(parent)]=res

print(str(parent_to_child_class_from_0_to_5))

11
[15, 19, 21, 31, 38]
Original key list is : [15, 19, 21, 31, 38]
Original value list is : [0, 1, 2, 3, 4]
15
[27, 29, 44, 78, 93]
Original key list is : [27, 29, 44, 78, 93]
Original value list is : [0, 1, 2, 3, 4]
4
[0, 51, 53, 57, 83]
Original key list is : [0, 51, 53, 57, 83]
Original value list is : [0, 1, 2, 3, 4]
14
[2, 11, 35, 46, 98]
Original key list is : [2, 11, 35, 46, 98]
Original value list is : [0, 1, 2, 3, 4]
1
[1, 32, 67, 73, 91]
Original key list is : [1, 32, 67, 73, 91]
Original value list is : [0, 1, 2, 3, 4]
5
[22, 39, 40, 86, 87]
Original key list is : [22, 39, 40, 86, 87]
Original value list is : [0, 1, 2, 3, 4]
18
[8, 13, 48, 58, 90]
Original key list is : [8, 13, 48, 58, 90]
Original value list is : [0, 1, 2, 3, 4]
3
[9, 10, 16, 28, 61]
Original key list is : [9, 10, 16, 28, 61]
Original value list is : [0, 1, 2, 3, 4]
10
[23, 33, 49, 60, 71]
Original key list is : [23, 33, 49, 60, 71]
Original value list is : [0, 1, 2, 3, 4]
17
[47, 52, 56, 59, 96]
Original 

In [21]:
parent_to_child_class_from_0_to_5[3]

{9: 0, 10: 1, 16: 2, 28: 3, 61: 4}

In [24]:
import json

with open("parent_to_child_class_from_0_to_5.json", "w") as outfile: 
    json.dump(parent_to_child_class_from_0_to_5, outfile)

In [25]:
f = open('parent_to_child_class_from_0_to_5.json')
data = json.load(f)
print(data)

{'11': {'15': 0, '19': 1, '21': 2, '31': 3, '38': 4}, '15': {'27': 0, '29': 1, '44': 2, '78': 3, '93': 4}, '4': {'0': 0, '51': 1, '53': 2, '57': 3, '83': 4}, '14': {'2': 0, '11': 1, '35': 2, '46': 3, '98': 4}, '1': {'1': 0, '32': 1, '67': 2, '73': 3, '91': 4}, '5': {'22': 0, '39': 1, '40': 2, '86': 3, '87': 4}, '18': {'8': 0, '13': 1, '48': 2, '58': 3, '90': 4}, '3': {'9': 0, '10': 1, '16': 2, '28': 3, '61': 4}, '10': {'23': 0, '33': 1, '49': 2, '60': 3, '71': 4}, '17': {'47': 0, '52': 1, '56': 2, '59': 3, '96': 4}, '2': {'54': 0, '62': 1, '70': 2, '82': 3, '92': 4}, '9': {'12': 0, '17': 1, '37': 2, '68': 3, '76': 4}, '8': {'3': 0, '42': 1, '43': 2, '88': 3, '97': 4}, '16': {'36': 0, '50': 1, '65': 2, '74': 3, '80': 4}, '6': {'5': 0, '20': 1, '25': 2, '84': 3, '94': 4}, '12': {'34': 0, '63': 1, '64': 2, '66': 3, '75': 4}, '19': {'41': 0, '69': 1, '81': 2, '85': 3, '89': 4}, '7': {'6': 0, '7': 1, '14': 2, '18': 3, '24': 4}, '13': {'26': 0, '45': 1, '77': 2, '79': 3, '99': 4}, '0': {'4':

In [28]:
data['0']

print(type('0'))

<class 'str'>


In [16]:

test_keys = [4, 30, 55, 72, 95]
test_values = [0, 1, 2, 3 ,4]

# Printing original keys-value lists
print("Original key list is : " + str(test_keys))
print("Original value list is : " + str(test_values))

# using naive method
# to convert lists to dictionary
res = {}
for key in test_keys:
    for value in test_values:
        res[key] = value
        test_values.remove(value)
        break

print(str(res))

Original key list is : [4, 30, 55, 72, 95]
Original value list is : [0, 1, 2, 3, 4]
{4: 0, 30: 1, 55: 2, 72: 3, 95: 4}


In [6]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [7]:
device = get_default_device()
device

device(type='cuda')

In [8]:
trainloader = DeviceDataLoader(train_loader, device)
testloader = DeviceDataLoader(test_loader, device)

In [9]:
import torch.nn as nn
import torch.nn.functional as F

In [11]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

class ImageClassificationBase(nn.Module):
    def __init__(self, fine):
        super(ImageClassificationBase, self).__init__()
        self.fine = fine
        #self.parent_to_child_class={}
        self.unique_outs=[]
    
    def training_step(self, batch):
        images, fine, coarse = batch
        if self.fine:
            labels=fine
        else:
            labels=coarse
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, fine, coarse = batch 

        #print("coarse ", coarse)
        #print("fine ", fine)    
        if self.fine:
            labels=fine
        else:
            labels=coarse
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], last_lr: {:.5f}, train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['lrs'][-1], result['train_loss'], result['val_loss'], result['val_acc']))
        
def conv_block(in_channels, out_channels, pool=False):
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1), 
              nn.BatchNorm2d(out_channels), 
              nn.ReLU(inplace=True)]
    if pool: layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)

class ResNet9(ImageClassificationBase):
    def __init__(self, in_channels, num_classes, fine):
        super().__init__(fine)
        
        self.conv1 = conv_block(in_channels, 64)
        self.conv2 = conv_block(64, 128, pool=True) 
        self.res1 = nn.Sequential(conv_block(128, 128), conv_block(128, 128)) 
        
        self.conv3 = conv_block(128, 256, pool=True)
        self.conv4 = conv_block(256, 512, pool=True) 
        self.res2 = nn.Sequential(conv_block(512, 512), conv_block(512, 512)) 
        self.conv5 = conv_block(512, 1028, pool=True) 
        self.res3 = nn.Sequential(conv_block(1028, 1028), conv_block(1028, 1028))  
        
        self.classifier = nn.Sequential(nn.MaxPool2d(2), # 1028 x 1 x 1
                                        nn.Flatten(), # 1028 
                                        nn.Linear(1028, num_classes)) # 1028 -> 100
        
    def forward(self, xb):
        out = self.conv1(xb)
        out = self.conv2(out)
        out = self.res1(out) + out
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.res2(out) + out
        out = self.conv5(out)
        out = self.res3(out) + out
        out = self.classifier(out)
        parent=out.argmax(dim=1, keepdim=True)
        print(torch.flatten(parent).cpu().numpy())
        print(parent.shape)
        #delete=delete.cpu().numpy().argmax()
        #if delete not in self.unique_outs:
        #    self.unique_outs.append(delete)
        #print(self.unique_outs)
        return out

#model100 = to_device(ResNet9(3, 100, True), device)
#model100


In [12]:
@torch.no_grad()
def evaluate(model, test_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in test_loader]
    return model.validation_epoch_end(outputs)

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def fit_one_cycle(epochs, max_lr, model, train_loader, test_loader, 
                  weight_decay=0, grad_clip=None, opt_func=torch.optim.SGD):
    torch.cuda.empty_cache()
    history = []
    
    # Set up cutom optimizer with weight decay
    optimizer = opt_func(model.parameters(), max_lr, weight_decay=weight_decay)
    # Set up one-cycle learning rate scheduler
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs, 
                                                steps_per_epoch=len(train_loader))
    
    for epoch in range(epochs):
        # Training Phase 
        model.train()
        train_losses = []
        lrs = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            
            # Gradient clipping
            if grad_clip: 
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)
            
            optimizer.step()
            optimizer.zero_grad()
            
            # Record & update learning rate
            lrs.append(get_lr(optimizer))
            sched.step()
        
        # Validation phase
        result = evaluate(model, test_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        result['lrs'] = lrs
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [13]:
import time
from sklearn.metrics import *

In [14]:
# Generate testing accuracy, predicted label, confusion matrix, and table for classification report
def test_label_predictions(model, device, test_loader, fine):
    model.eval()
    actuals = []
    predictions = []
    with torch.no_grad():
        if fine:
            for data, target, _ in test_loader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                prediction = output.argmax(dim=1, keepdim=True)
                print("predicted ", prediction, " actual: ", target)
                actuals.extend(target.view_as(prediction))
                predictions.extend(prediction)
        else:
            for data, _, target in test_loader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                prediction = output.argmax(dim=1, keepdim=True)
                print("predicted ", prediction, " actual: ", target)
                actuals.extend(target.view_as(prediction))
                predictions.extend(prediction)

    return [i.item() for i in actuals], [i.item() for i in predictions]

In [15]:
from torchsummary import summary

#print(summary(model100,(3, 32, 32)))

-----------------------------------------------------------------------------------

In [16]:
model20 = to_device(ResNet9(3, 20, False), device)
model20

ResNet9(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (conv2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (res1): Sequential(
    (0): Sequential(
      (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (1): Sequential(
      (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=Tr

In [17]:
# Initial evaluation
history = evaluate(model20, testloader)


[8 8 9 8 8 4 8 8 8 4 8 8 8 8 4 9 8 4 4 8 4 4 8 8 8 8 8 8 8 8 8 4 8 8 4 8 8
 8 8 8 8 8 4 8 8 4 8 8 8 8 8 8 8 8 8 8 9 8 9 8 4 9 8 4 8 8 4 8 4 8 8 8 4 9
 8 8 8 8 8 8 8 8 8 4 8 8 8 9 4 9 8 8 8 8 8 4 8 4 8 8 8 8 8 8 4 8 8 8 8 8 4
 8 8 8 8 8 4 8 8 8 8 4 4 4 8 8 8 8 4 8 8 8 9 8 8 8 8 8 8 8 8 8 8 8 8 9 8 8
 4 8 4 4 8 8 4 8 8 8 8 8 4 4 4 8 8 8 4 8 8 8 8 8 8 8 8 8 8 8 9 8 8 4 8 9 4
 4 8 8 8 4 8 8 8 8 8 8 8 8 4 8 8 4 8 8 8 4 8 8 8 8 8 8 8 8 4 9 8 9 8 8 8 8
 8 8 4 8 8 8 8 8 8 8 8 8 8 4 4 8 8 8 8 8 4 8 8 8 8 8 9 8 4 8 9 8 8 8 8 9 4
 4 8 9 8 8 8 8 8 4 8 8 8 8 8 8 8 4 9 4 8 8 8 8 4 8 9 8 8 8 4 8 4 8 8 8 8 8
 8 4 8 8 4 8 8 8 8 4 8 8 8 8 4 9 8 8 4 8 8 4 8 8 9 8 8 8 8 8 4 4 8 8 8 8 8
 8 8 8 8 4 4 8 8 8 8 8 8 8 8 8 8 4 8 9 8 8 8 8 8 8 8 8 8 8 8 9 9 8 8 8 8 8
 8 8 8 8 8 8 8 8 8 4 4 4 8 8 8 8 8 8 8 8 8 8 8 8 4 8 8 8 8 8 8 8 8 8 8 8 8
 8 8 8 8 8 8 8 4 8 4 8 4 8 8 8 8 8 8 4 4 8 9 8 8 4 9 8 4 8 8 8 8 4 8 8 9 8
 8 8 8 8 8 8 8 8 8 4 8 8 4 8 8 8 8 8 8 8 8 8 8 8 8 8 8 4 4 8 8 9 8 8 9 8 8
 8 8 8 8 8 8 4 4 8 8 8 8 

In [42]:
y_test, y_pred = test_label_predictions(model20, device, testloader, False)
cm=confusion_matrix(y_test, y_pred)
cr=classification_report(y_test, y_pred)
fs=f1_score(y_test,y_pred,average='weighted')
rs=recall_score(y_test, y_pred,average='weighted')
accuracy=accuracy_score(y_test, y_pred)
print('Confusion matrix:')
print(cm)
print(cr)
print('F1 score: %f' % fs)
print('Recall score: %f' % rs)
print('Accuracy score: %f' % accuracy)

predicted  tensor([[1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
 

/home/arsen.abzhanov/.conda/envs/Copy_of_CV701_env_for_Temporal_Santosh/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/arsen.abzhanov/.conda/envs/Copy_of_CV701_env_for_Temporal_Santosh/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/arsen.abzhanov/.conda/envs/Copy_of_CV701_env_for_Temporal_Santosh/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samp

In [ ]:
from torchsummary import summary

print(summary(model20,(3, 32, 32)))

-----------------------------------------------------------------------------------

In [24]:
model20To100 = ResNet9(3, 20, True)
model20To100.load_state_dict(torch.load('group20_pretrained_model.h5'))
for param in model20To100.parameters():
    param.requires_grad=False

#model20To100.classifier=nn.Sequential(model20To100.classifier,nn.Linear(20, 100))
#model20To100=to_device(model20To100, device)
#model20To100

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [19]:
model20To100=to_device(model20To100, device)

In [22]:
# Initial evaluation
history = evaluate(model20To100, testloader)


RuntimeError: Caught RuntimeError in pin memory thread for device 0.
Original Traceback (most recent call last):
  File "/home/arsen.abzhanov/.conda/envs/Copy_of_CV701_env_for_Temporal_Santosh/lib/python3.8/site-packages/torch/utils/data/_utils/pin_memory.py", line 34, in do_one_step
    data = pin_memory(data, device)
  File "/home/arsen.abzhanov/.conda/envs/Copy_of_CV701_env_for_Temporal_Santosh/lib/python3.8/site-packages/torch/utils/data/_utils/pin_memory.py", line 70, in pin_memory
    return type(data)([pin_memory(sample, device) for sample in data])  # type: ignore[call-arg]
  File "/home/arsen.abzhanov/.conda/envs/Copy_of_CV701_env_for_Temporal_Santosh/lib/python3.8/site-packages/torch/utils/data/_utils/pin_memory.py", line 70, in <listcomp>
    return type(data)([pin_memory(sample, device) for sample in data])  # type: ignore[call-arg]
  File "/home/arsen.abzhanov/.conda/envs/Copy_of_CV701_env_for_Temporal_Santosh/lib/python3.8/site-packages/torch/utils/data/_utils/pin_memory.

In [ ]:
# Fitting the first 1/4 epochs
current_time=time.time()
history += fit_one_cycle(int(epochs/4), max_lr, model20To100, trainloader, testloader, 
                             grad_clip=grad_clip, 
                             weight_decay=weight_decay, 
                             opt_func=opt_func)

In [ ]:
# Fitting the second 1/4 epochs
history += fit_one_cycle(int(epochs/4), max_lr/10, model20To100, trainloader, testloader, 
                             grad_clip=grad_clip, 
                             weight_decay=weight_decay, 
                             opt_func=opt_func)

In [ ]:
# Saving the model to h5 file
torch.save(model20To100.state_dict(), 'group20To100_pretrained_model.h5')

In [ ]:
y_test, y_pred = test_label_predictions(model20To100, device, testloader)
cm=confusion_matrix(y_test, y_pred)
cr=classification_report(y_test, y_pred)
fs=f1_score(y_test,y_pred,average='weighted')
rs=recall_score(y_test, y_pred,average='weighted')
accuracy=accuracy_score(y_test, y_pred)
print('Confusion matrix:')
print(cm)
print(cr)
print('F1 score: %f' % fs)
print('Recall score: %f' % rs)
print('Accuracy score: %f' % accuracy)

In [ ]:
print(summary(model20To100,(3, 32, 32)))